Utilizaremos el archivo de nombre “A1.4 Vino Tinto.csv”, donde podremos encontrar información
para 1,599 observaciones distintas, con 11 mediciones para cada una de ellas, así como con una
variable de salida, la calidad asignada a dicho vino. Los datos se descargaron del UCI Machine
Learning Repository, y originalmente se reportaron en una publicación científica para la revista
Decision Support Systems.

A continuación, generaremos un modelo de regresión lineal múltiple que contenga solamente
las variables seleccionadas por un proceso de selección hacia adelante y eliminación hacia atrás.


1. Importaremos los datos del archivo “Vino Tinto.csv” al ambiente de trabajo. Revisaremos las dimensiones
del data frame e imprime en consola tanto dichas dimensiones como las primeras 5 filas
de datos.


In [22]:
import pandas as pd
df = pd.read_csv('A1.4 Vino Tinto.csv')
df_shape = df.shape
df_head = df.head()
df_shape, df_head

#df.describe()

((1599, 12),
    acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
 0         7.4           0.70          0.00             1.9     0.076   
 1         7.8           0.88          0.00             2.6     0.098   
 2         7.8           0.76          0.04             2.3     0.092   
 3        11.2           0.28          0.56             1.9     0.075   
 4         7.4           0.70          0.00             1.9     0.076   
 
    dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
 0                11.0                34.0    0.9978  3.51      0.56      9.4   
 1                25.0                67.0    0.9968  3.20      0.68      9.8   
 2                15.0                54.0    0.9970  3.26      0.65      9.8   
 3                17.0                60.0    0.9980  3.16      0.58      9.8   
 4                11.0                34.0    0.9978  3.51      0.56      9.4   
 
    calidad  
 0        5  
 1        5  
 2        5  
 3 

2. Separaremos el data frame en datos de entrenamiento y datos de prueba con una proporción
80/20. Es decir, el 80% de los datos se usarán para entrenar el modelo y el resto para
validar los resultados. La partición será aleatoria.Imprimiremos en pantalla las dimensiones de ambos conjuntos de datos. 

In [26]:
from sklearn.model_selection import train_test_split

# Separamos las variables independientes (X) de la variable dependiente (y)
X = df.drop('calidad', axis=1)  # Variables independientes (sin la columna 'calidad')
y = df['calidad']  # Variable dependiente (calidad)

# Realizamos la división de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((1279, 11), (320, 11), (1279,), (320,))

3. Generaremos la metodología de selección hacia adelante e imprimiremos en consola los índices o
los nombres de las características seleccionadas.A continuación se muestra la descripción de los
parámetros que que es recomendado usar:


In [30]:
pip install mlxtend


Note: you may need to restart the kernel to use updated packages.


a. estimator. Usaremos la función
LinearRegression de la librería sklearn.linear_model, para generar la regresión lineal.


b. k_features. Se puede seleccionar la cantidad de variables de salida que se desean,
pero te recomiendo mejor usar un rango, y que el algoritmo determine el número
adecuado. Por ejemplo, puedes definir el parámetro como (2,8), si te interesa que
el método seleccione entre 2 y 8 variables.

c. forward. Determina si se hace selección hacia adelante (True) o hacia atrás (False);
en este caso queremos hacer selección hacia adelante = True.


d. scoring. La métrica que se usará para determinar si un modelo es mejor que otro,
te recomiendo definirla como ‘r2’ para usar la R cuadrada.


e. cv. Si se desea realizar validación cruzada, y cuántas instancias de la misma. Te
recomiendo definir este parámetro como 10.

In [35]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Modelo de regresión lineal
estimator = LinearRegression()

# Definimos el selector de características hacia adelante
sfs = SFS(estimator,
          k_features=(2, 8),        # Selección entre 2 y 8 características
          forward=True,             # Selección hacia adelante
          floating=False,           # No flotante, es decir, seleccionamos un número fijo de características
          scoring='r2',             # Métrica de rendimiento R cuadrada
          cv=10)                    # Validación cruzada con 10 divisiones

# Ajustamos el selector a los datos de entrenamiento
sfs.fit(X_train, y_train)

# Imprimimos las características seleccionadas
print("Índices de características seleccionadas:", sfs.k_feature_idx_)
print("Nombres de características seleccionadas:", X_train.columns[list(sfs.k_feature_idx_)])


Índices de características seleccionadas: (1, 2, 4, 6, 8, 9, 10)
Nombres de características seleccionadas: Index(['acidezVolatil', 'acidoCitrico', 'cloruros', 'dioxidoAzufreTotal', 'pH',
       'sulfatos', 'alcohol'],
      dtype='object')


Nos podemos percatar que las carácterísticas de mayor impacto en el modelo de acuerdo a la metodología utilizadason: 'acidezVolatil', 'acidoCitrico', 'cloruros', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol'.

4. Ahora entrenaremos un modelo que solamente contenga las variables seleccionadas, predecir la
respuesta en las observaciones de prueba y medir la capacidad de predicción del modelo
usando la R cuadrada, imprimiendo dicho valor en consola.  

In [43]:
from sklearn.metrics import r2_score

# Seleccionamos las columnas correspondientes a las características seleccionadas en los datos de entrenamiento y prueba
X_train_selected = X_train.iloc[:, list(sfs.k_feature_idx_)]  
X_test_selected = X_test.iloc[:, list(sfs.k_feature_idx_)]  

model = LinearRegression()

# Entrenamos el modelo con las características seleccionadas
model.fit(X_train_selected, y_train)

# Realizamos predicciones sobre el conjunto de prueba con las características seleccionadas
y_pred = model.predict(X_test_selected)

# Calculamos el R cuadrada 
r2 = r2_score(y_test, y_pred)

# Imprimimos el valor de R cuadrada
print("Valor de R cuadrada (R²) del modelo:", r2)


Valor de R cuadrada (R²) del modelo: 0.35194540293487375


5. Realizaremos un proceso de selección hacia atrás a partir de las variables seleccionadas por
el método de selección hacia adelante e imprimir en consola los índices o nombres de las
variables seleccionadas.

In [45]:
# Realizamos la selección hacia atrás a partir de las variables seleccionadas por el método hacia adelante
sfs_backward = SFS(estimator,
                   k_features=(2, 5),         # Selección entre 2 y 5 características
                   forward=False,             # Selección hacia atrás (False)
                   floating=False,            # No flotante
                   scoring='r2',              # Métrica de rendimiento R cuadrada
                   cv=10)                     # Validación cruzada con 10 pliegues

# Ajustamos el selector a las características seleccionadas por el método hacia adelante
sfs_backward.fit(X_train_selected, y_train)


print("Índices de características seleccionadas por selección hacia atrás:", sfs_backward.k_feature_idx_)
print("Nombres de características seleccionadas por selección hacia atrás:", X_train_selected.columns[list(sfs_backward.k_feature_idx_)])


Índices de características seleccionadas por selección hacia atrás: (0, 2, 4, 5, 6)
Nombres de características seleccionadas por selección hacia atrás: Index(['acidezVolatil', 'cloruros', 'pH', 'sulfatos', 'alcohol'], dtype='object')


6. Repetiremos el paso 4, pero para un modelo que contenga solamente las variables
seleccionadas en el paso 5. 

In [49]:
# Seleccionamos las columnas correspondientes a las características seleccionadas en la selección hacia atrás para el conjunto de entrenamiento y prueba
X_train_backward = X_train.iloc[:, list(sfs_backward.k_feature_idx_)]  
X_test_backward = X_test.iloc[:, list(sfs_backward.k_feature_idx_)]  

model_backward = LinearRegression()

# Entrenamos el modelo con las características de selección hacia atrás.
model_backward.fit(X_train_backward, y_train)

# Realizamos predicciones sobre el conjunto de prueba.
y_pred_backward = model_backward.predict(X_test_backward)


r2_backward = r2_score(y_test, y_pred_backward)
print("Valor de R cuadrada (R²) del modelo con selección hacia atrás:", r2_backward)


Valor de R cuadrada (R²) del modelo con selección hacia atrás: 0.16547305770943854


En este caso, la selección hacia adelante parece ser la mejor opción para este problema en particular, ya que permite mantener un conjunto de características que mejora el ajuste del modelo a los datos. Sin embargo, es importante recordar que R² es solo una medida del rendimiento, y podría ser útil explorar otras métricas o técnicas, como la validación cruzada o la regularización, para obtener un modelo más robusto.
